# Aufgabe 1: Rucksackproblem

In [110]:
def solve_backpack(weight_list, value_list, max_i, max_weight):
    """
    optimize the value given a maximum weight by choosing several items from the lists
    inputs:
        weight_list - (list) - list of positive floats representing weights
        value_list - (list) - list of positive floats representing value
            both lists must have equal length
    returns:
        (float) - maximum value of the chosen items
        (list) of ints - indices of the chosen items
    """
    if max_i < 0: #no items left to choose
        return 0, []
    if max_weight < 0: #
        return -1e10, []
    vi, wi = value_list[max_i], weight_list[max_i]
    # calculate sub-problems:
    # don't choose the current item
    value_2, items_2 = solve_backpack(weight_list, value_list, max_i-1, max_weight)
    if wi > max_weight:
        return value_2, items_2
    # choose the current item:
    value_1, items_1 = solve_backpack(weight_list, value_list, max_i-1, max_weight-wi)
    value_1 += vi
    # if the value of choosing the current item is greater than the value for not choosing it, return the value and list for when the item is chosen.
    if value_1 > value_2:
        return value_1, items_1+[max_i]
    return value_2, items_2

In [111]:
# Vorlesungsbeispiel:
import random
pairs = [(1,15), (5,10), (3,9), (4,5)]
random.shuffle(pairs)
weigths, values = list(zip(*pairs))
print("weigths: ", weigths)
print("values:  ", values)
print(*solve_backpack(weigths, values, len(weigths)-1, 8))

weigths:  (3, 4, 1, 5)
values:   (9, 5, 15, 10)
29 [0, 1, 2]


In [112]:
weigths = [5, 4, 6, 3]
values = [10, 40, 30, 50]
print(*solve_backpack(weigths, values, len(weigths)-1, 12))

100 [0, 1, 3]
